# データの読み込み

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#必要なライブラリのインポート
import numpy as np
from tensorflow.keras import Input,Model
from tensorflow.keras.layers import Conv2D,BatchNormalization,MaxPooling2D,concatenate,Conv2DTranspose,Add,Dense,Dropout,Flatten
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Precision, Recall,Accuracy
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.metrics import classification_report,precision_score,roc_curve,f1_score,roc_auc_score,auc,confusion_matrix,classification_report
from keras.optimizers import Adam
#ファイルの呼び出し
a=np.load('/content/drive/MyDrive/Colab Notebooks/4lab/data/npz&npy/256px,max/No Finding_array_5000.npy')
a=a[:2500]
a1=np.load('/content/drive/MyDrive/Colab Notebooks/4lab/data/npz&npy/256px,max/Atelectasis_array.npy')
a2=np.load('/content/drive/MyDrive/Colab Notebooks/4lab/data/npz&npy/256px,max/Cardiomegaly_array.npy')
a3=np.load('/content/drive/MyDrive/Colab Notebooks/4lab/data/npz&npy/256px,max/Effusion_array.npy')
a4=np.load('/content/drive/MyDrive/Colab Notebooks/4lab/data/npz&npy/256px,max/Infiltration_array.npy')
a5=np.load('/content/drive/MyDrive/Colab Notebooks/4lab/data/npz&npy/256px,max/Mass_array.npy')
a6=np.load('/content/drive/MyDrive/Colab Notebooks/4lab/data/npz&npy/256px,max/Nodule_array.npy')
a8=np.load('/content/drive/MyDrive/Colab Notebooks/4lab/data/npz&npy/256px,max/Pneumothorax_array.npy')
x=np.concatenate([a,a1,a2,a3,a4,a5,a6,a8],axis=0)
No_Finding_label=np.array([0]*len(a))
Disease_label1=np.array([1]*len(a1))
Disease_label2=np.array([2]*len(a2))
Disease_label3=np.array([3]*len(a3))
Disease_label4=np.array([4]*len(a4))
Disease_label5=np.array([5]*len(a5))
Disease_label6=np.array([6]*len(a6))
Disease_label7=np.array([7]*len(a8))
y=np.concatenate([No_Finding_label,Disease_label1,Disease_label2,Disease_label3,Disease_label4,Disease_label5,Disease_label6,Disease_label7],axis=0)


In [ ]:
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import time

#画像データおよびラベルの 8 割を訓練用、2 割をテスト用に分ける
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20,random_state=0)
#画像データの正規化を行う
x_train=x_train.astype('float32')/255
x_test=x_test.astype('float32')/255

# PyTorchで扱うため、tensor型にする
x_train= torch.tensor(x_train)
x_test= torch.tensor(x_test)
y_train= torch.tensor(y_train)
y_test= torch.tensor(y_test)
#NCWH形式にする
x_train= x_train.permute(0,3,1,2)
x_test= x_test.permute(0,3,1,2)

# 画像データ・正解ラベルのペアをデータにセットする
train_dataset = torch.utils.data.TensorDataset(x_train, y_train)
test_dataset = torch.utils.data.TensorDataset(x_test, y_test)
# セットしたデータをバッチサイズごとの配列に入れる。
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)#バッチサイズでメモリ消費変わる
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# DualNetの作成(DualVGG16)

In [ ]:
import torch
import torch.nn as nn


class Block1(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Block1, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )

    def forward(self, x):
        return self.layers(x)
class Block2(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Block2, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )

    def forward(self, x):
        return self.layers(x)
class Classifier(nn.Module):
    def __init__(self, in_features, num_classes):
        super(Classifier, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        return self.layers(x)

class DualVGG16(nn.Module):
    def __init__(self, num_classes):
        super(DualVGG16, self).__init__()
        self.s1_extractor = self._make_extractor()
        self.s2_extractor = self._make_extractor()
        self.fused_classifier = self._make_classifier()
        self.s1_classifier = self._make_classifier()
        self.s2_classifier = self._make_classifier()

    def _make_extractor(self):
        extractor = nn.Sequential(
            Block1(1, 64),
            Block1(64, 128),
            Block2(128, 256),
            Block2(256, 512),
            Block2(512, 512),
            nn.Dropout(0.5),
            nn.Flatten()
        )
        return extractor

    def _make_classifier(self):
        return Classifier(512*8*8, num_classes)

    def forward(self, x):
        s1_features = self.s1_extractor(x)
        s2_features = self.s2_extractor(x)
        fused_features = s1_features + s2_features

        fused_output = self.fused_classifier(fused_features)
        s1_output = self.s1_classifier(s1_features)
        s2_output = self.s2_classifier(s2_features)
        return fused_output, s1_output, s2_output

    def check_cnn_size(self, size_check):
        out = self.conv_layers(size_check)

        return out

# モデルのインスタンスを作成
num_classes = 8
model = DualVGG16(num_classes)
device = torch.device("cuda")
# modelはnn.Moduleを継承したクラス
model = model.to(device) # GPUへ転送

# 学習

In [ ]:
import torch.nn.functional as F
from tensorflow.python.ops.gen_math_ops import TV_NotEqual_T
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import time
'''評価用の関数を定義'''
def valid(test_loader):

    running_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            fused_output, s1_output, s2_output = model(images)
            outputs=fused_output+0.3*s1_output+0.3*s2_output
            outputs=F.softmax(outputs,dim=1)
            loss_fused = criterion(fused_output, labels)
            loss_s1 = criterion(s1_output, labels)
            loss_s2 = criterion(s2_output, labels)
            total_loss = loss_fused + 0.3*loss_s1 + 0.3*loss_s2
            running_loss += total_loss.item()
            predicted = outputs.max(1, keepdim=True)[1]
            labels = labels.view_as(predicted)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)
    val_loss = running_loss / len(test_loader)
    val_acc = correct / total
    return val_loss, val_acc

device = torch.device("cuda")

# 損失関数
criterion = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(model.parameters(), lr=0.0001)
train_loss_value=[]       #trainのlossを保持するlist
train_acc_value=[]        #trainのaccuracyを保持するlist
test_loss_value=[]       #testのlossを保持するlist
test_acc_value=[]        #testのaccuracyを保持するlist
# 学習
num_epochs =15
for epoch in range(num_epochs):
    start = time.time()
    train_loss = 0
    train_acc = 0
    train_batches=0
    model.train()
    if epoch%2==0:
      # パラメータ固定
      for param in model.s1_extractor.parameters():
        param.required_grad=False
      for param in model.s1_classifier.parameters():
        param.required_grad=False

      for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()# 勾配を初期化
        #output
        fused_output, s1_output, s2_output = model(images) # 順伝播
        outputs=fused_output+0.3*s1_output+0.3*s2_output
        outputs=F.softmax(outputs,dim=1)
        #loss
        loss_fused = criterion(fused_output, labels)
        loss_s2 = criterion(s2_output, labels)
        total_loss = loss_fused + 0.3*loss_s2
        train_batches+=1
        #結果を保存
        train_loss += total_loss.item()
        train_acc += (outputs.max(1)[1] == labels).sum().item()
        #逆伝搬
        total_loss.backward()
        optimizer.step()
      #パラメータを戻す
      for param in model.s1_extractor.parameters():
        param.required_grad=True
      for param in model.s1_classifier.parameters():
        param.required_grad=True
    else :
      # パラメータ固定
      for param in model.s2_extractor.parameters():
        param.required_grad=False
      for param in model.s2_classifier.parameters():
        param.required_grad=False

      for images, labels in train_loader:
          images = images.to(device)
          labels = labels.to(device)
          optimizer.zero_grad()# 勾配を初期化
          #output
          fused_output, s1_output, s2_output = model(images) # 順伝播
          outputs=fused_output+0.3*s1_output+0.3*s2_output
          outputs=F.softmax(outputs,dim=1)
          #loss
          loss_fused = criterion(fused_output, labels)
          loss_s1 = criterion(s1_output, labels)
          total_loss = loss_fused + 0.3*loss_s1
          train_batches+=1
          #結果を保存
          train_loss += total_loss.item()
          train_acc += (outputs.max(1)[1] == labels).sum().item()
          #逆伝搬
          total_loss.backward()
          optimizer.step()
      #パラメータを戻す
      for param in model.s2_extractor.parameters():
        param.required_grad=True
      for param in model.s2_classifier.parameters():
        param.required_grad=True
    #訓練結果の保存
    avg_train_loss = train_loss / train_batches
    avg_train_acc = train_acc / len(train_loader.dataset)
    train_loss_value.append(avg_train_loss)
    train_acc_value.append(avg_train_acc)
    #テスト
    model.eval()
    avg_val_loss,avg_val_acc=valid(test_loader)
    #テスト結果の保存
    test_loss_value.append(avg_val_loss)
    test_acc_value.append(avg_val_acc)
    end = time.time()
    # ログの表示
    print('Epoch [{}/{}], train_loss: {train_loss:.4f} train_acc: {train_acc:.4f} val_loss: {val_loss:.4f}, val_acc: {val_acc:.4f},time: {time:.4f}'
                   .format(epoch+1, num_epochs, train_loss=avg_train_loss,train_acc=avg_train_acc, val_loss=avg_val_loss, val_acc=avg_val_acc,time=end-start))

# モデルを保存する。
#torch.save(model, "/content/drive/MyDrive/Colab Notebooks/4lab/code/model/Dual_VGG16(5000-2).pth")
torch.save(model.state_dict(), '/content/drive/MyDrive/Colab Notebooks/4lab/code/model/DualNet_VGG16_weights(3000).pth')
plt.figure(figsize=(6,6))      #グラフ描画用
#以下グラフ描画
plt.plot(range(1,num_epochs+1), train_loss_value)
plt.plot(range(1,num_epochs+1), test_loss_value, c='#00ff00')
plt.xlim(0, num_epochs+1)
plt.ylim(0, 5)
plt.xlabel('EPOCH')
plt.ylabel('LOSS')
plt.legend(['train loss','test loss'])
plt.title('loss')
plt.plot()

plt.figure(figsize=(6,6))
plt.plot(range(1,num_epochs+1), train_acc_value)
plt.plot(range(1,num_epochs+1), test_acc_value, c='#00ff00')
plt.xlim(0, num_epochs+1)
plt.ylim(0, 1)
plt.xlabel('EPOCH')
plt.ylabel('ACCURACY')
plt.legend( ['train acc','test acc'])
plt.title('accuracy')
plt.plot()


# AUC Scoreの表示

In [ ]:

import torch.nn.functional as F
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
model.load_state_dict(torch.load( '/content/drive/MyDrive/Colab Notebooks/4lab/code/model/DualNet_VGG16_weights(3000).pth'))
model.eval()

predictions = []
actuals = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        fused_output, s1_output, s2_output = model(inputs)
        outputs = fused_output+0.3*s1_output+0.3*s2_output
        predictions.extend(outputs.cpu().numpy())
        actuals.extend(labels.cpu().numpy())


def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / e_x.sum(axis=-1, keepdims=True)

Disease_label=[0,1,2,3,4,5,6,7]
Disease_name=['No_Finding','Atelectasis','Cardiomegaly','Effusion','Infiltration','Mass','Nodule','Pneumothorax']
actuals_one_hot = label_binarize(actuals, classes=Disease_label)
predictions = np.array(predictions)　
predictions = softmax(predictions)



n_classes = len(Disease_label)
fpr = {}
tpr = {}
roc_auc = {}
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(actuals_one_hot[:, i], predictions[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(6,6))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=Disease_name[i])
plt.legend()


from sklearn.metrics import auc
c=0
for i in range(n_classes):
    print(Disease_name[i],":",auc(fpr[i],tpr[i]))
    c+=auc(fpr[i],tpr[i])
print('overall',c/8)
plt.plot()

# おまけ

## DualVGG19

In [ ]:
import torch
import torch.nn as nn



class Block1(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Block1, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )

    def forward(self, x):
        return self.layers(x)

class Block2(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Block2, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2)
        )

    def forward(self, x):
        return self.layers(x)

class Classifier(nn.Module):
    def __init__(self, in_features, num_classes):
        super(Classifier, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        return self.layers(x)

class DualVGG19(nn.Module):
    def __init__(self, num_classes):
        super(DualVGG19, self).__init__()
        self.s1_extractor = self._make_extractor()
        self.s2_extractor = self._make_extractor()
        self.fused_classifier = self._make_classifier()
        self.s1_classifier = self._make_classifier()
        self.s2_classifier = self._make_classifier()

    def _make_extractor(self):
        extractor = nn.Sequential(
            Block1(1, 64),
            Block1(64, 128),
            Block2(128, 256),
            Block2(256, 512),
            Block2(512, 512),
            nn.Dropout(0.5),
            nn.Flatten()
        )
        return extractor

    def _make_classifier(self):
        return Classifier(512*8*8, num_classes)

    def forward(self, x):
        s1_features = self.s1_extractor(x)
        s2_features = self.s2_extractor(x)
        fused_features = s1_features + s2_features

        fused_output = self.fused_classifier(fused_features)
        s1_output = self.s1_classifier(s1_features)
        s2_output = self.s2_classifier(s2_features)
        return fused_output, s1_output, s2_output

    def check_cnn_size(self, size_check):
        out = self.conv_layers(size_check)

        return out

# モデルのインスタンスを作成
num_classes = 8
model = DualVGG19(num_classes)
device = torch.device("cuda")
# modelはnn.Moduleを継承したクラス
model = model.to(device) # GPUへ転送

## DualResNet(50)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import time

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(
        in_channels,
        out_channels,
        kernel_size=3,
        stride=stride,
        padding=1,
        bias=False,
    )

def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(
        in_channels, out_channels, kernel_size=1, stride=stride, bias=False
    )
#bottleneckブロックは1×1,3x3、1x1の3つの畳み込み層から構成されている。
#expansionは～
class Bottleneck(nn.Module):
    expansion = 4  # 出力のチャンネル数を入力のチャンネル数の何倍に拡大するか

    def __init__(self, in_channels, channels, stride=1):
        super().__init__()
        self.conv1 = conv1x1(in_channels, channels)
        self.bn1 = nn.BatchNorm2d(channels)
        self.conv2 = conv3x3(channels, channels, stride)
        self.bn2 = nn.BatchNorm2d(channels)
        self.conv3 = conv1x1(channels, channels * self.expansion)
        self.bn3 = nn.BatchNorm2d(channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        # 入力と出力のチャンネル数が異なる場合、x をダウンサンプリングする。
        if in_channels != channels * self.expansion:
            self.shortcut = nn.Sequential(
                conv1x1(in_channels, channels * self.expansion, stride),
                nn.BatchNorm2d(channels * self.expansion),
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += self.shortcut(x)

        out = self.relu(out)

        return out


class Classifier(nn.Module):
    def __init__(self, in_features, num_classes):
        super(Classifier, self).__init__()
        self.fc =nn.Linear(in_features*4, num_classes)

    def forward(self, x):
        return self.fc(x)

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=8):
        super().__init__()
        self.s1_extractor = self._make_extractor(block,)
        self.s2_extractor = self._make_extractor()
        self.fused_classifier = self._make_classifier()
        self.s1_classifier = self._make_classifier()
        self.s2_classifier = self._make_classifier()
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # 重みを初期化する。（Heの初期化法）
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)



    def _make_layer(self, block, channels, blocks, stride):
        layers = []

        # 最初の Residual Block
        layers.append(block(self.in_channels, channels, stride))

        # 残りの Residual Block
        self.in_channels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, channels))

        return nn.Sequential(*layers)

    def _make_extractor(self,block,channels, blocks, stride):
        self.in_channels = 64#self.conv1を通過した後のチャンネル
        extractor=nn.Sequential(
            conv1 = nn.Conv2d(
            1, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False
            ),
            bn1 = nn.BatchNorm2d(self.in_channels),
            relu = nn.ReLU(inplace=True),
            maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            layer1 = self._make_layer(block, 64, layers[0], stride=1),
            layer2 = self._make_layer(block, 128, layers[1], stride=2),
            layer3 = self._make_layer(block, 256, layers[2], stride=2),
            layer4 = self._make_layer(block, 512, layers[3], stride=2),
            avgpool = nn.AdaptiveAvgPool2d((1, 1))
        )
        return extractor
    def _make_classifier(self):
        return Classifier(512,num_classes)


    def forward(self, x):
        s1_features = self.s1_extractor(x)
        s2_features = self.s2_extractor(x)
        fused_features = s1_features + s2_features
        fused_output = self.fused_classifier(fused_features)
        s1_output = self.s1_classifier(s1_features)
        s2_output = self.s2_classifier(s2_features)
        return fused_output, s1_output, s2_output


def resnet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])



# モデルのインスタンス化
num_classes = 8
custom_model = resnet50()
device = torch.device("cuda")
custom_model=custom_model.to(device)

